# Introduction: Testing the Spatial-KWD library
In this notebook, we present a short tutorial on the use of the python wrapper for the **Spatial-KWD** software library.

The goal of this notebook is to show how to use Spatial-KWD to compute the distance between pairs of 2-dimensional histograms.

The only requirements for running this notebook are:

1.   A recent version of [Cython](https://cython.org/) (>= v0.23) installed on your PC.
2.   To Download the source code of the Spatial-KWD python wrapper from [EUROSTAT public repository](https://github.com/eurostat/Spatial-KWD/tree/main/wrappers/python).





In [ ]:
import shutil
import sys

# Install Cython
if not shutil.which("cython"):
  !pip install -q cython
  assert(shutil.which("cython"))

# Get the Spatial-KWD source files
if "google.colab" in sys.modules:
  !wget -N -q "https://github.com/eurostat/Spatial-KWD/archive/v0.1.5-alpha.tar.gz"
  !tar -zxf v0.1.5-alpha.tar.gz

In [ ]:
%cd Spatial-KWD-0.1.5-alpha/wrappers/python
!ls 

Once you have the requirements ready, you can compile the library, with the following command

In [ ]:
!python setup.py build_ext --inplace

# Helper functions
With the following example you can test the three helper functions:
 * `compareOneToOne(Coordinates, Weights, Options)`: Compute the KW distance between a pair of histograms, given in the two columns of matrix `Weights`.
 * `compareOneToMany(Coordinates, Weights, Options)`: Compute the KW distance between a reference histogram (first column in `Weights`) and a set of other histograms (remaining columns of `Weights`).
 * `compareAll(Coordinates, Weights, Options)`: Compute the KW distance between all histograms in `Weights`.

In [ ]:
import numpy as np
# Import helper functions from S-KWD
from KWD import compareOneToOne, compareOneToMany, compareAll

np.random.seed(13)

N = 100
M = 3

# Random data
Coordinates = np.random.randint(0, 32, size=(N, 2), dtype=np.int32)
Weights = np.random.uniform(0, 100, size=(N, 2))

# Testing helper functions
print('-----------------------------\nTest one2one approx:')
Options = {}
sol = compareOneToOne(Coordinates, Weights, Options)
for k in sol:
    print(k, sol[k])
print()

print('-----------------------------\nTest one2one exact:')
Options = {'Method': 'exact' }
sol = compareOneToOne(Coordinates, Weights, Options)
for k in sol:
    print(k, sol[k])
print()

print('-----------------------------\nTest one2many approx:')
Weights = np.random.uniform(0, 100, size=(N, M))
Options = {}
sol = compareOneToMany(Coordinates, Weights, Options)
for k in sol:
    print(k, sol[k])
print()

print('-----------------------------\nTest all approx:')
Weights = np.random.uniform(0, 100, size=(N, M))
Options = {}
sol = compareAll(Coordinates, Weights, Options)
for k in sol:
    print(k, sol[k])

### DEPRECATED, TO BE FIXED: Histogram2D
The Spatial-KWD library contains a main class of type `Histogram2D` that is used to store a 2-dimensional histogram.
Note that the histogram does not need to be *rectangular*, but it can be defined by specifing the bins coordinates of the elements with a strictly positive weight.

This class has three main methods:

1. A standard costructor `h = Histogram2D()` 
2. A method to add a single point with positive weight: `h.add(int x, int y, double weight)`
3. A method to *normalize* the bin weights, so that the sum of all weights is equal to 1: `h.normalize()`

We now show to use this class to build three different histograms.

In [ ]:
from KWD import *

# Define first histogram
#a = Histogram2D()
# Add at position (0,0) a unit of mass
#a.add(0, 0, 1)
# Normalize the histogram
#a.normalize()

# Define second histogram
#b = Histogram2D()
# Add at position (1,1) two units of mass
#b.add(1, 1, 2)
# Normalize the histogram
#b.normalize()

These two dummy histograms have a single bin with positive weight. The second histogram is normalized and the effect is to reduce its weight to 1.

Then, we add a third histogram having three non empty bins. Please, note that we are not using a regular grid to define this histogram. You are free to position the points at any pair of integer coordinates.

In [ ]:
# Define third histogram
#c = Histogram2D()
# Add at position (1,0) and (0,1) an half unit of mass
#c.add(1, 0, 0.5)
#c.add(0, 1, 0.5)
# Add at position (5,5) a unit of mass
#c.add(5, 5, 1)
# Normalize the histogram
#c.normalize()

After normalizarion, the weights of the three bins become 0.25, 0.25, 0.5.

We show in the next section how to compute the distances among these three histograms. 

### Using the Solver to compute hte KWD distances
The Spatial-KWD library has another object called `Solver` which is used to configure the algorithm that computes the distance between a pair of histograms.

In [ ]:
# Create an instance of type Solver 
#s = Solver()

Finally we can compute the distance among the three histograms using the method `s.distance(Histogram2D h1, Histogram2D h1, L=3)`.
The method distance first creates a cloud of points considering the union of non empty bins of the two histograms, second it computes the convex hull of all this cloud of points, and finally it computes the Kantorovich-Wasserstein distances between the two histograms using the algorithm described in [[1]](https://epubs.siam.org/doi/abs/10.1137/19M1261195).

The third parameter `L` is used to specify the approximation quality of the algorithm, and it will be described in detail in the second tutorial on this library.


In [ ]:
#print("d(a,b) =", s.distance(a, b, 3))
#print("d(a,c) =", s.distance(a, c, 3))
#print("d(b,c) =", s.distance(b, c, 3))

Please, if you have any comment on this notebook or on the Spatial-KWD you can drop me an email.

The source code of the library is freely available on GitHub at [Eurostat/Spatial-KWD](https://github.com/eurostat/Spatial-KWD).

### References

[1] Bassetti F., Gualandi S., Veneroni M. (2018): [**On the computation of Kantorovich-Wasserstein distances between 2D-histograms by uncapacitated minimum cost flows**](https://epubs.siam.org/doi/abs/10.1137/19M1261195). SIAM J. Optim., 30(3), 2441–2469, 2020. Preprint on arXiv: [1804.00445](https://arxiv.org/abs/1804.00445).